<a href="https://colab.research.google.com/github/kaifkh20/nn_z_to_h/blob/main/makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
from google.colab import files
uploaded = files.upload()

Saving names(1).txt to names(1) (1).txt


In [62]:
import torch
import torch.nn.functional as F
words = open("names(1).txt").read().splitlines()

In [63]:
#character mapping
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [64]:
#building the dataset

block_size = 3 #context length
X,Y = [],[]

for w in words:
  # print(w)
  context = [0] * block_size #pad it with ...
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

In [66]:
#build training, dev and test dataset

def build_dataset(words):
  block_size = 3 #context length
  X,Y = [],[]

  for w in words:
    # print(w)
    context = [0] * block_size #pad it with ...
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]
  X = torch.tensor(X)
  Y = torch.tensor(Y)

  return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


# Example word: "emma"
# X shape: [32,3] means we have 32 training examples, each with 3 letters of context
# Each number in X represents a letter (0-26 for a-z)

# One row in X might look like:
# [4, 12, 12]  # representing "emm" (context to predict "a")


C = torch.randn(27, 2)  # 27 letters (a-z + .), each gets 2 numbers
# C gives each letter a unique "coordinate" in 2D space
# Example:
# 'e' (index 4) might get coordinate [-0.1, 0.5]
# 'm' (index 12) might get coordinate [0.3, -0.2]

# If X has one row [4, 12, 12] ("emm")
# emb will convert it to:
# [[-0.1, 0.5],   # coordinates for 'e'
#  [0.3, -0.2],   # coordinates for 'm'
#  [0.3, -0.2]]   # coordinates for 'm'

In [46]:
C = torch.randn((27,2))
emb = C[X]
#emb[:,0] # so suppopse we have three input layer as seen in paper then this for the first layer and for first word

In [47]:
W1 = torch.randn(6,100) # we get the 6 from emb[32,3,2] = 6
b1 = torch.randn(100)

we do now torch.cat operation which is means that the layer is fed with all three words parameters at once into all the neurons

a context have total of 3 letters each having 2 position
which in total having 6 position of a context

[0,
 1,
 2]  --> [0,1,2,3,4,5...100]

In [48]:
#this is one way but ineffecient -> torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]],1).shape

why 6 since we are predicting the next token based on three sized block context with each token have two dimension.


In [49]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1) #(-1,6) means pytorch will figure what should be its value
h.shape

torch.Size([228146, 100])

Last layer (soft max classifications)

In [50]:
W2 = torch.randn(100,27)
b2 = torch.randn(27)

parameters = [C,W1,b1,W2,b2]


logits = h @ W2 + b2
#this is for education purpose

# counts = logits.exp()
# probs = counts / counts.sum(1,keepdims=True)
# loss = -probs[torch.arange(32),Y].log().mean() #negative likeli-hood

#actually we should do it like this

loss = F.cross_entropy(logits,Y)


In [67]:
for p in parameters:
  p.requires_grad = True

referring above for every details now we tidy all the things up

In [77]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn([27,2],generator=g)
W1 = torch.randn([6,300],generator=g)
b1 = torch.randn(300,generator=g)
W2 = torch.randn([300,27],generator=g)
b2 = torch.randn(27,generator=g)
parameters = [C,W1,b1,W2,b2]

for p in parameters:
  p.requires_grad = True

In [78]:
#this is to find a good learning rate
lrs = torch.linspace(-3,0,1000)
lre = 10**lrs #10^-3 and so on

In [79]:
lri = []
lossi = []
for i in range (10000):

  #minibatch construct
  ix = torch.randint(0,Xtr.shape[0],(32,))

  #forward pass
  emb = C[Xtr[ix]]
  h = torch.tanh(emb.view(-1,6) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits,Ytr[ix])
  # print(loss.item())
  #backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  #update
  # lr = lrs[i]
  lr = 0.1
  for p in parameters:
    p.data += -lr * p.grad

    #track status
    #lri.append(lre[i])
    #lossi.append(loss.item())
print(loss.item())

2.8747668266296387


search about learning rate decay

In [80]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,Ytr)

print(loss.item())

2.650872230529785


In [81]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,Ydev)

print(loss.item())

2.646641731262207


In [56]:
#training split, dev/validation split , test split
# 80% , 10% , 10%

Watch the video after 1:00 to understand on how to improve the nn

In [ ]:
#sample from model

g = torch.Generator().manual_seed(2147483647+30)
for _ in range(20):
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
        emb = torch.tensor([context]) # (1,block_size,d)
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = torch.softmax(logits, dim=-1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))